# Problem Formulation

Data isn't clean and it has missing information,imbalanced,noise.In addition to dataset contains columns have more than one feature and contain words with numbers so we need to make preprocessing for this dataset.
After we make preprocessing, data will be cleaned for making prediction.

# The input and output

The input represents as variable X that contains(total 22 columns)

In [1]:
#x.columns

The output that represents as y (target = rating)

# What data mining function is required?

Association analysis aims to discover associations between items occurring together frequently.              
prediction based on testing data by making models such as (desicion tree and support vector machine and naive bayes)  

# What could be the challenges?

Firstly choose the right and optimal columns that has effect on the target and how can you split column and take the interest value 
,represent the difficult words as simple word or value to be easily for understanding

# What is the impact?

The impact of dataset is the performance including support vector machine (SVM), decision tree (DT), adaboost (AB), and naïve Bayes (NB)
,analyze the performance of the models when trained on each resulting dataset with respect to f1-score

# What is an ideal solution?

ideal best solution will be the maximum value is decision tree

# Importing libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# read data

In [3]:
data_train=pd.read_csv('train_new.csv')
#data_train.columns
data_test=pd.read_csv('test_new.csv') #read test data

In [4]:
datad1=data_train.drop(columns='id')


# Make feature selection

In [5]:
label_encoder = preprocessing.LabelEncoder()
for i in datad1:
    datad1[i]=label_encoder.fit_transform(datad1[i])

In [6]:
X=datad1.loc[:,datad1.columns!='rating']
Y=datad1.rating
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
bestfeatures = SelectKBest(score_func=chi2, k=20)
fit = bestfeatures.fit(X,Y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores

,0,0
0,price,143.041698
1,retail_price,515.263995
2,currency_buyer,NaN
3,units_sold,50.771618
4,uses_ad_boosts,3.769768
5,rating_count,15721.092091
6,badges_count,95.427215
7,badge_local_product,0.568173
8,badge_product_quality,106.191687
9,badge_fast_shipping,87.402612


# Data exploration


In [7]:
# display information od data
print(data_train.head(20))
data_train.info() #print all column headers + type
data_train.shape #number of rows and columns
data_train.isna().sum() #check null value for every feature
#data_train['product_variation_size_id'].unique()
#data_train['product_variation_size_id'].value_counts().head(30)
data_test.info()

    price  retail_price currency_buyer  units_sold  uses_ad_boosts  rating  \
0    8.00             7            EUR       10000               1     4.0   
1    8.00             7            EUR        1000               0     4.0   
2    6.00            34            EUR         100               0     4.0   
3    4.67             4            EUR         100               1     3.0   
4    8.00            11            EUR        1000               0     4.0   
5   14.00            51            EUR         100               0     4.0   
6    3.67             9            EUR        5000               0     3.0   
7    8.00             7            EUR          50               1     4.0   
8    9.00             8            EUR        5000               0     4.0   
9   19.00            21            EUR        5000               0     4.0   
10  12.00            11            EUR        1000               1     4.0   
11   5.74             5            EUR       10000              

# Data cleaning and pre-processing

In [8]:
data_train.drop(columns=['merchant_has_profile_picture','merchant_profile_picture','currency_buyer','tags'
                         , 'shipping_is_express', 'urgency_text', 'merchant_name'
                         , 'merchant_id', 'theme', 'crawl_month'],inplace=True)
#drop columns that don't effect on rating (target) to increase accuracy
#inplace= to show changes that i make in data
data_test.drop(columns=['merchant_has_profile_picture','merchant_profile_picture','currency_buyer','tags'
                         , 'shipping_is_express', 'urgency_text', 'merchant_name'
                         , 'merchant_id', 'theme', 'crawl_month'],inplace=True)

# Transform (merchant_rating) from numbers with decimal point into numbers without decimal numbers to be optimal for rating because we didn't see any rate with 4.81 before


In [9]:
def map_hours(x):
    if x>5.0:
        return 6.0
    elif x > 4.0 :
        return 5.0
    elif x > 3.0 and x < 4.0:
        return 4.0
    elif x > 2.0 and  x < 3.0:
        return 3.0
    elif x > 1.0 and  x < 2.0:
        return 2.0
    elif x > 0.0 and  x<1.0:
        return 1.0
    else:
        return 0.0


In [10]:
data_train['merchant_rating'] = data_train['merchant_rating'].apply(map_hours)

In [11]:
#data_train['merchant_info_subtitle']=data_train['merchant_info_subtitle'].map(lambda x: x[:2])


In [12]:
data_train['product_color'].unique()

array(['yellow', 'black', 'white', 'lakeblue', 'apricot', 'brown',
       'winered', 'blue', 'red', 'navyblue', 'green', 'khaki', 'White',
       'white & green', 'multicolor', 'lightpink', 'pink', 'RED',
       'armygreen', 'lightblue', nan, 'coffee', 'grey', 'skyblue',
       'watermelonred', 'pink & black', 'whitefloral', 'purple', 'navy',
       'pink & white', 'rosered', 'orange', 'Black', 'mintgreen',
       'leopardprint', 'gray', 'navy blue', 'star', 'rose', 'lightyellow',
       'camouflage', 'black & yellow', 'whitestripe', 'navyblue & white',
       'black & blue', 'lightred', 'violet', 'gold', 'black & green',
       'white & black', 'burgundy', 'black & white', 'lightgrey',
       'coolblack', 'lightgreen', 'beige', 'darkblue', 'darkgreen',
       'silver', 'wine red', 'Army green', 'pink & blue', 'rainbow',
       'claret', 'floral', 'brown & yellow', 'light green', 'Pink',
       'blue & pink', 'dustypink', 'camel', 'orange-red', 'rosegold',
       'ivory', 'fluorescentg

In [13]:
data_train['product_color'].value_counts().head(20)

black            219
white            173
blue              77
yellow            69
pink              68
green             61
red               61
grey              48
purple            37
armygreen         22
winered           19
navyblue          18
orange            16
multicolor        14
khaki             10
white & green      8
lightblue          8
coffee             6
gray               6
skyblue            6
Name: product_color, dtype: int64

In [14]:
# choose the basic colour and map it with another color 
color={'white':1,'whitefloral':1,'whitestripe':1,'white & black':1,
       'offwhite':1,'yellow':2,'khaki':12,'lightyellow':2,'black':3,'Black':3,'coolblack':3
      ,'blue':4,'lakeblue':4,'navyblue':4,'lightblue':4,'skyblue':4,'Blue':4,'pink':5
       ,'rosered':5,'rose':5,'floral':5,'rosegold':5,'lightpink':5,'green':6
       ,'armygreen':6,'mintgreen':6,'lightgreen':6,'applegreen':6,'darkgreen':6
       ,'light green':6,'fluorescentgreen':6,'red':7,'RED':7,'winered':11,'Rose red':7,'lightred':7
      ,'grey':8,'lightgrey':8,'gray':8,'silver':8,'purple':9,'orange':10,'orange-red':10,'white & green':13,'multicolor':14}

In [15]:
data_train['product_color']= data_train['product_color'].map(color)
data_train

,price,retail_price,units_sold,uses_ad_boosts,rating,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,...,shipping_option_price,countries_shipped_to,inventory_total,has_urgency_banner,origin_country,merchant_title,merchant_info_subtitle,merchant_rating_count,merchant_rating,id
0,8.00,7,10000,1,4.0,1670,0,0,0,0,...,3,30,50,NaN,CN,keepahorse,"88 % avis positifs (66,644 notes)",66644,5.0,272
1,8.00,7,1000,0,4.0,843,1,1,0,0,...,3,37,50,NaN,CN,shanghaimingqidianqikejiyouxiangongsi,"91 % avis positifs (25,752 notes)",25752,5.0,1515
2,6.00,34,100,0,4.0,10,0,0,0,0,...,2,41,50,NaN,CN,zhaodongmei,"83 % avis positifs (3,897 notes)",3897,4.0,940
3,4.67,4,100,1,3.0,82,0,0,0,0,...,1,25,50,NaN,CN,pookie0331,"87 % avis positifs (7,497 notes)",7497,5.0,309
4,8.00,11,1000,0,4.0,127,0,0,0,0,...,3,15,50,1.0,CN,shitongyi12683626,91 % avis positifs (173 notes),173,5.0,226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,4.00,4,10000,0,4.0,2150,0,0,0,0,...,1,41,50,NaN,CN,fantastic_tune,"92 % avis positifs (230,951 notes)",230951,5.0,1381
1090,5.00,5,1000,0,5.0,377,1,0,1,0,...,2,35,50,NaN,CN,zzbllss,"(2,904 notes)",2904,5.0,212
1091,9.00,48,1000,1,4.0,135,0,0,0,0,...,3,25,50,NaN,CN,OIUG 56465,"78 % avis positifs (6,632 notes)",6632,4.0,163
1092,11.00,10,10000,1,4.0,1479,0,0,0,0,...,2,45,50,1.0,CN,yubenqiang,"83 % avis positifs (5,816 notes)",5816,4.0,674


In [16]:
data_train.isna().sum()

price                            0
retail_price                     0
units_sold                       0
uses_ad_boosts                   0
rating                           0
rating_count                     0
badges_count                     0
badge_local_product              0
badge_product_quality            0
badge_fast_shipping              0
product_color                  105
product_variation_size_id       12
product_variation_inventory      0
shipping_option_name             0
shipping_option_price            0
countries_shipped_to             0
inventory_total                  0
has_urgency_banner             745
origin_country                  12
merchant_title                   0
merchant_info_subtitle           0
merchant_rating_count            0
merchant_rating                  0
id                               0
dtype: int64

In [17]:
#map the same size_id together 
size_dic={'S':10,'S Pink':10,'s':10,'Size-S':10,'Size S':10,'S.':10,'Size S.':10,'size S':10,'25-S':10
          ,'S..':10,'pants-S':10,'Size/S':10,'Suit-S':10
          ,'Size/S':10,'SIZE S':10,'S (waist58-62cm)':10,'25-S':10,'XS':1,'SIZE XS':1,'XS.':1,'Size-XS':1
,'S/M(child)':10,'M':2,'M.':2,'Size M':2,'XXS':3,'Size-XXS':3,'SIZE XXS':3
,'L':4,29:4,'XL':5,'X   L':5,'Women Size 36':5,'XXL':6,'XXXS':7,'XXXS':7,'XXXXL':8,'4XL':8,'Size4XL':8
          ,'5XL':9,'Size-5XL':9,'XXXXXL':9}

In [18]:
data_train['product_variation_size_id']= data_train['product_variation_size_id'].map(size_dic)
data_train['product_variation_size_id']

0        2.0
1        4.0
2        4.0
3        1.0
4       10.0
        ... 
1089    10.0
1090     NaN
1091     2.0
1092     2.0
1093     1.0
Name: product_variation_size_id, Length: 1094, dtype: float64

In [19]:
data_test['product_variation_size_id']= data_test['product_variation_size_id'].map(size_dic)
data_test['product_color']= data_test['product_color'].map(color)
data_test['merchant_rating'] = data_test['merchant_rating'].apply(map_hours)


# Label Encoding handels categorical variables,each label is assigned a unique integer based on alphabetical ordering.

In [20]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'origin_country'. 
data_train['origin_country']= label_encoder.fit_transform(data_train['origin_country'])
#Encode labels in column 'shipping_option_name'.
data_train['shipping_option_name']= label_encoder.fit_transform(data_train['shipping_option_name'])
#Encode labels in column 'merchant_title'.
data_train['merchant_title']= label_encoder.fit_transform(data_train['merchant_title'])
#Encode labels in column 'merchant_info_subtitle'.
data_train['merchant_info_subtitle']= label_encoder.fit_transform(data_train['merchant_info_subtitle'])


# For testing data

In [21]:
data_test['origin_country']= label_encoder.fit_transform(data_test['origin_country'])
data_test['shipping_option_name']= label_encoder.fit_transform(data_test['shipping_option_name'])
data_test['merchant_title']= label_encoder.fit_transform(data_test['merchant_title'])
data_test['merchant_info_subtitle']= label_encoder.fit_transform(data_test['merchant_info_subtitle'])
data_train.isna().sum() #check null value for every feature


price                            0
retail_price                     0
units_sold                       0
uses_ad_boosts                   0
rating                           0
rating_count                     0
badges_count                     0
badge_local_product              0
badge_product_quality            0
badge_fast_shipping              0
product_color                  105
product_variation_size_id       68
product_variation_inventory      0
shipping_option_name             0
shipping_option_price            0
countries_shipped_to             0
inventory_total                  0
has_urgency_banner             745
origin_country                   0
merchant_title                   0
merchant_info_subtitle           0
merchant_rating_count            0
merchant_rating                  0
id                               0
dtype: int64

# fill the missing values

In [22]:
data_train['origin_country'].fillna(method='ffill')
data_train['product_color']=data_train['product_color'].fillna(0)
data_train['product_variation_size_id']=data_train['product_variation_size_id'].fillna(0)
data_train['has_urgency_banner']=data_train['has_urgency_banner'].fillna(0)

In [23]:
data_test['origin_country'].fillna(method='ffill')
data_test['product_color']=data_test['product_color'].fillna(0)
data_test['product_variation_size_id']=data_test['product_variation_size_id'].fillna(0)
data_test['has_urgency_banner']=data_test['has_urgency_banner'].fillna(0)


# Select input and output

In [24]:
#id is not a feature to be selected
data_test.dropp=data_test.drop(columns='id')
datad=data_train.drop(columns='id') #drop id 
x=datad.loc[:,datad.columns!='rating'] # X is all the features after preprocessing without (rating)
y=datad.rating #target
data_test.dropp

/var/folders/wz/v4hv9zdx6bz0lyd2gqbp5hy40000gn/T/ipykernel_1638/1426719615.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_test.dropp=data_test.drop(columns='id')


,price,retail_price,units_sold,uses_ad_boosts,rating_count,badges_count,badge_local_product,badge_product_quality,badge_fast_shipping,product_color,...,shipping_option_name,shipping_option_price,countries_shipped_to,inventory_total,has_urgency_banner,origin_country,merchant_title,merchant_info_subtitle,merchant_rating_count,merchant_rating
0,5.75,5,10,1,0,0,0,0,0,1.0,...,4,2,139,50,0.0,1,250,80,3,3.0
1,7.00,7,5000,0,579,0,0,0,0,3.0,...,4,2,31,50,0.0,1,321,261,4213,5.0
2,5.00,16,1000,1,62,0,0,0,0,4.0,...,4,2,41,50,0.0,1,114,206,10501,5.0
3,18.00,27,5000,1,1903,0,0,0,0,4.0,...,4,5,42,50,0.0,1,286,249,15855,5.0
4,12.00,81,20000,0,1799,0,0,0,0,1.0,...,4,3,38,50,0.0,1,187,314,151914,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,16.00,14,100,1,27,0,0,0,0,3.0,...,4,4,35,50,0.0,3,48,82,572,4.0
475,6.00,17,5000,1,817,0,0,0,0,4.0,...,4,2,41,50,0.0,1,41,298,50864,5.0
476,4.83,51,50,1,1,0,0,0,0,1.0,...,4,1,39,50,1.0,3,307,232,57,4.0
477,3.00,3,100,1,8,0,0,0,0,3.0,...,4,1,26,50,1.0,1,252,274,1086,5.0


# change all columns with int data type to float data type

In [25]:
int_col = data_train.select_dtypes(include=['int64']) # This will select float columns only
 # list(float_col.columns.values)

for col in int_col.columns.values:
    data_train[col] = data_train[col].astype('float64')

# data_train['merchant_rating']=data_train['merchant_rating'].astype('int64')

In [26]:
int_col = data_test.select_dtypes(include=['int64']) # This will select float columns only
 # list(float_col.columns.values)
for col in int_col.columns.values:
    data_test[col] = data_test[col].astype('float64')
data_test['id']=data_test['id'].astype('int64')

In [27]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   price                        479 non-null    float64
 1   retail_price                 479 non-null    float64
 2   units_sold                   479 non-null    float64
 3   uses_ad_boosts               479 non-null    float64
 4   rating_count                 479 non-null    float64
 5   badges_count                 479 non-null    float64
 6   badge_local_product          479 non-null    float64
 7   badge_product_quality        479 non-null    float64
 8   badge_fast_shipping          479 non-null    float64
 9   product_color                479 non-null    float64
 10  product_variation_size_id    479 non-null    float64
 11  product_variation_inventory  479 non-null    float64
 12  shipping_option_name         479 non-null    float64
 13  shipping_option_pric

# Decision Tree Model

In [28]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(random_state=42)

# Train Decision Tree Classifer
clf = clf.fit(x,y)

# By using hyperparameter

In [29]:
model_af=DecisionTreeClassifier(criterion= "gini", max_depth= 7 , min_samples_leaf= 3,random_state=42 )
#model_af.fit(x,y)
#AdaBoost classifier builds a strong classifier by combining multiple poorly performing classifiers so that you will get high accuracy strong classifier
#abc1 =AdaBoostClassifier(n_estimators=50, base_estimator=model_af,learning_rate=1)
#base_estimator:It uses DecisionTreeClassifier for training purpose
#n_estimators: Number of weak learners to train iteratively.
#learning_rate: It contributes to the weights of weak learners. It uses 1 as a default value.
model_af1=model_af.fit(x,y)
y_pred_af = model_af1.predict(data_test.dropp)
pred_df0=pd.DataFrame(data={'id':np.asarray(data_test['id']),'rating':y_pred_af})
pred_df0.to_csv('pred_walthroughafter.csv',index=False)


# Tuning hyperparameter by using GridSearchCV
grid search is a method to search best hyperparameters exhaustively from the grid of given hyperparameters

In [30]:
# add many of hyperparameters
param_dist = {
     "criterion" : ["gini" , "entropy"],
     "max_depth" : [2 , 4 , 6 , 8, 10, 12, 14, 16],
     "min_samples_leaf": [1 , 2, 3, 4, 5, 6, 7, 8, 9, 10]
 }
#"criterion":The function to measure the quality of a split. 
#Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain.
#"max_depth":The maximum depth of the tree.
#"min_samples_leaf":The minimum number of samples required to be at a leaf node.
# list of dictionaries
GS = GridSearchCV(clf , param_grid= param_dist , cv= 20 , n_jobs= -1)
# n_jobs(Number of jobs to run in parallel , -1 means using all processors)
# cv (Determines the cross-validation splitting strategy)
GS.fit(x,y)
print(GS.best_params_)
print(GS.best_score_)

/Users/sondosmostafa/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=20.
  warnings.warn(("The least populated class in y has only %d"


{'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 1}
0.7822895622895623


AdaBoost classifier builds a strong classifier by combining multiple poorly performing classifiers so that you will get high accuracy strong classifier

In [31]:
model = DecisionTreeClassifier(criterion= "gini", max_depth= 4 , min_samples_leaf= 1 , random_state= 42)
#model.fit(x , y)

#Create adaboost classifer object
#abc =AdaBoostClassifier(n_estimators=50, base_estimator=model,learning_rate=1)

# Train Classifer
model1 = model.fit(x,y)

#Predict the response for test dataset
y_pred = model1.predict(data_test.dropp)

In [32]:
pred_df=pd.DataFrame(data={'id':np.asarray(data_test['id'].astype('int64')),'rating':y_pred})
pred_df.to_csv('pred_walthroughDeecisiontreee3.csv',index=False)


In [33]:
pred_df

,id,rating
0,937,5.0
1,270,4.0
2,308,4.0
3,913,4.0
4,403,4.0
...,...,...
474,1166,4.0
475,1416,4.0
476,63,2.0
477,626,4.0


# SVM classifier

In [34]:
model_svc = SVC(random_state=42)
model_svc.fit(x, y)

SVC(random_state=42)

# first hyperparameters

In [35]:
model_svc1=SVC(probability=True,C= 1000, gamma= 0.01, kernel= 'rbf' ,random_state=42)
model_svc1.fit(x,y)
predict_svc1=model_svc1.predict(data_test.dropp)
pred_df1=pd.DataFrame(data={'id':np.asarray(data_test['id']),'rating':predict_svc1})
# pred_df1.to_csv('pred_walthroughsvm1.csv',index=False)

# Second hyperparameters

In [36]:
clf2=GridSearchCV(model_svc, {
        'kernel': ['rbf'],
        'gamma': [0.1,0.01,0.001,0.001],
         'C':[0.01,10,1.0,100,1000,1e5]
    }, 
    cv= 10 , n_jobs= -1
                 )
#C:Regularization parameter. 
#The strength of the regularization is inversely proportional to C. Must be strictly positive.


In [37]:
clf2.fit(x, y)

/Users/sondosmostafa/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=10, estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.01, 10, 1.0, 100, 1000, 100000.0],
                         'gamma': [0.1, 0.01, 0.001, 0.001],
                         'kernel': ['rbf']})

In [38]:
print(clf2.best_params_)
print(clf2.best_score_)

{'C': 1.0, 'gamma': 0.001, 'kernel': 'rbf'}
0.7659966638865722


In [39]:
svc=SVC(probability=True,C= 1, gamma= 0.001, kernel= 'rbf' ,random_state=42)

# # Create adaboost classifer object
# abc_svm =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)
# #base_estimator:It uses SVC for training purpose
# # Train Classifer
model_svm = svc.fit(x, y)
# #Predict the response for test dataset
#evaluate your model’s accuracy 
y_pred_svm = model_svm.predict(data_test.dropp)

In [40]:
pred_df1=pd.DataFrame(data={'id':np.asarray(data_test['id']),'rating':y_pred_svm})
pred_df1.to_csv('pred_walthroughsvm.csv',index=False)

In [41]:
 #pred_df1

# Naive Bayes
Theorem:provides a way that we can calculate the probability of a piece of data belonging to a given class             
naive bayes classifier : assumes that there is independent relationship between features
There is no hyperparameter tune in naive bayes

In [42]:
from sklearn.naive_bayes import BernoulliNB
clf_naive = BernoulliNB()
clf_naive.fit(x,y)
y_pred_naive = clf_naive.predict(data_test.dropp)
y_pred_naive

array([5., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5.,
       4., 4., 5., 4., 5., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 5., 5., 4., 4., 5., 6., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 5.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 4., 4., 4., 4., 5., 5., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5.,
       4., 4., 5., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 4., 4., 4., 4., 4., 4.,
       4., 5., 4., 4., 4.

In [43]:
pred_df2=pd.DataFrame(data={'id':np.asarray(data_test['id']),'rating':y_pred_naive})
pred_df2.to_csv('pred_walthroughnaive.csv',index=False)

# Merge two datasets two check F1-score

In [44]:
# merge two datasets to evalute models before , i submit in kaggle
merged_df= pd.concat([data_train, data_test], axis=0)

In [45]:
merged_df.isna().sum()

price                            0
retail_price                     0
units_sold                       0
uses_ad_boosts                   0
rating                         479
rating_count                     0
badges_count                     0
badge_local_product              0
badge_product_quality            0
badge_fast_shipping              0
product_color                    0
product_variation_size_id        0
product_variation_inventory      0
shipping_option_name             0
shipping_option_price            0
countries_shipped_to             0
inventory_total                  0
has_urgency_banner               0
origin_country                   0
merchant_title                   0
merchant_info_subtitle           0
merchant_rating_count            0
merchant_rating                  0
id                               0
dtype: int64

In [46]:
merged_df['rating']=merged_df['rating'].fillna(0)
merged_df1=merged_df.drop(columns='id')

In [47]:
merged_df1['product_variation_size_id'].head(20)

0      2.0
1      4.0
2      4.0
3      1.0
4     10.0
5     10.0
6      5.0
7     10.0
8      2.0
9      0.0
10     4.0
11     2.0
12    10.0
13     5.0
14    10.0
15     0.0
16     1.0
17     1.0
18     1.0
19    10.0
Name: product_variation_size_id, dtype: float64

In [48]:
x1=merged_df1.loc[:,datad.columns!='rating'] # X is all the features after preprocessing without (rating)
y1=merged_df1.rating

In [49]:
xtr,xts,ytr,yts=train_test_split(x1,y1,test_size=0.2,random_state=42)

In [50]:
param_dist = {
     "criterion" : ["gini" , "entropy"],
     "max_depth" : [2 , 4 , 6 , 8, 10, 12, 14, 16],
     "min_samples_leaf": [1 , 2, 3, 4, 5, 6, 7, 8, 9, 10]
 } # list of dictionaries
GS = GridSearchCV(DecisionTreeClassifier(random_state= 42 ) , param_grid= param_dist , cv= 10 , n_jobs= -1)
# n_jobs(Number of jobs to run in parallel , -1 means using all processors)
# cv (Determines the cross-validation splitting strategy)
GS.fit(xtr,ytr)
print(GS.best_params_)
print(GS.best_score_)

/Users/sondosmostafa/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


{'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 4}
0.826679365079365


In [51]:
model_af2=DecisionTreeClassifier(criterion= 'gini', max_depth= 4 , min_samples_leaf= 4, random_state= 42 )
#model_af2.fit(xtr,ytr)
#AdaBoost classifier builds a strong classifier by combining multiple poorly performing classifiers so that you will get high accuracy strong classifier
#abc1 =AdaBoostClassifier(n_estimators=50, base_estimator=model_af2,learning_rate=1)
#base_estimator:It uses DecisionTreeClassifier for training purpose
#n_estimators: Number of weak learners to train iteratively.
#learning_rate: It contributes to the weights of weak learners. It uses 1 as a default value.
model_af5=model_af2.fit(xtr,ytr)
y_pred_af1 = model_af5.predict(xts)

In [52]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xtr = sc.fit_transform(xtr)
xts = sc.transform(xts)

In [53]:
from sklearn.metrics import f1_score
print ('\n clasification report:\n', f1_score(yts,y_pred_af1,average='micro'))


 clasification report:
 0.8476190476190476


# Answers for questions

# 1-Why Data Mining is a misnomer? What is another preferred name?

-the goal of it is the extraction of patterns and knowledge from large amounts of data, not the extraction (mining) of data itself.

-Knowledge discovery in databases (KDD)

# 2-What is the general knowledge discovery process?                                                               
-process of extracting the important and interesting patterns or knowledge from huge amounts of data
Knowledge Discovery Process may consist of the following steps :

1-Data cleaning and preprocessing in which noise or outliers and inconsistent data is removed.
Strategies for handling missing data fields.

2-Data integration in which multiple data sources are combined.

3-Data selection or focusing on a subset of variables, or data samples, on which discovery is to be performed in which data relevant to the analysis task are retrieved from the database.

4-Data Transformation, data are transformed into forms appropriate for mining by performing summary or aggregation operations.

5-Data mining methods (algorithms) are applied in order to extract data patterns.

6-Pattern Evaluation, data patterns are identified based on some interesting measures.

7-Knowledge Presentation is represented to user using many knowledge representation techniques.

# What is the difference between a data engineer and data scientist/AI engineer?                                          
-Data scientists develop analytical models, while data engineers deploy those models in production. As such, data scientists focus primarily on analytics and solve business problems.
, and data engineers focus more heavily on programming , also prepare data for production by converting raw, unstructured data into a structured format that can be analyzed and interpreted. 
data engineers craft custom tools to support big data analytics. At large companies that handle large, complex volumes of data, data engineers often focus on optimizing ETL (extract-transform-load) processes.

# 3-In data mining, what is the difference between prediction and categorization?

1-categorization from classification is mostly based on our current or past assumptions,detecting the class label correctly,based on a training dataset.                        
2-Prediction is about predicting a missing/unknown element(continuous value) of a dataset,how well a given predictor can guess the value of a predicated attribute for new data,are supposed to done with both classification and regression for the test data set.

# 4- Why data science/machine learning is a bad idea in the context of information security?


when security measures such as access control fail,attackers can easily take the data and make use of any information it contains,so This mean data is availabled in unencrypted form . To make this more difficult, the data should always be stored in encrypted form. This way, the attacker
must steal the data when it is being processed or manage to steal the keys used to encrypt
it.

# What is CIA principle?



1-Confidentiality measures are designed to prevent sensitive information from unauthorized access attempts. It is common for data to be categorized according to the amount and type of damage that could be done if it fell into the wrong hands. 

2-Integrity involves maintaining the consistency, accuracy and trustworthiness of data over its entire lifecycle. Data shouldn't be changed in transport, and steps must be taken to ensure data cannot be changed by unauthorized people 

3-Availability means information should be consistently and readily accessible for authorized parties. This involves properly maintaining hardware and technical infrastructure and systems that hold and display the information.

# How can we use it to access the security/privacy aspect of the AI system/pipelines?

-for Confidentiality Data encryption is common method of ensuring confidentiality and make user ID with strong passwords for your accounts,Keep access control lists and other file permissions up to date.


-For integrity use backup and recovery software to restore the affected data to its correct state
To ensure integrity, use version control, access control, security control, data logs and checksums.

-For availability use preventive measures such as redundancy, failover .Systems and applications must be stayed updated.
-Use network or server monitoring systems.